# Options:

* generate a bag of words for each movie, possibly joining the director name words, and searching for the plot on imdb
* With this, we would create a keyword based system, with word vectors.




In [43]:
from collections import Counter, defaultdict
import pandas as pd
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from sklearn.externals import joblib

#Once
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
def process_user(filename):
    df = pd.read_csv(directory+"/"+filename)
    df = df.dropna(subset=['rated'])
    texts = []
    rates = []
    df['words'] = np.NaN
    df['recommend'] = np.NaN
    stop_words = stopwords.words('english')
    for index, row in df.iterrows():
        text = ""
        plot = row['Plot']
        directors = row.Directors
        writers = row.Writers
        genres = row.Genres
        rate = row.rated
        if not pd.isnull(directors): 
            dir_temp = directors.replace(" ","").lower().split("-")
            for dir in dir_temp:
                text = text + " " + dir
        if not pd.isnull(writers): 
            wri_temp = writers.replace(" ","").lower().split("-")
            for wri in wri_temp:
                text = text + " " + wri
        if not pd.isnull(genres): 
            gen_temp = genres.replace(" ","").lower().split("-")
            for gen in gen_temp:
                text = text + " " + gen

        if not pd.isnull(rate): 
            if rate < 4:
                rates.append("N")
                df.loc[index, 'recommend'] = "N"
            else:
                rates.append("Y")
                df.loc[index, 'recommend'] = "Y"
        if not pd.isnull(plot): 
            for r in plot.lower().split(" "): 
                if not r in stop_words:
                     text = text + " " + r
        texts.append(text)
        df.loc[index, 'words'] = text
    X = df['words']
    Y = df['recommend']
    clasificador_tfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
    ])
    if(len(Y.unique())>1):
        clasificador_tfidf.fit(X,Y)
        filename = "user_model/"+filename.replace(".csv","")
        joblib.dump(clasificador_tfidf,filename)
        print("saved file " + filename)

In [53]:
#We iterate over all files (1 for testing) in the data directory
directory = 'user_data'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        print("processing data for user " + filename.replace(".csv","") + "...")
        process_user(filename)


processing data for user 1050889...
saved file user_model/1050889
processing data for user 14756...
saved file user_model/14756
processing data for user 1989766...
saved file user_model/1989766
processing data for user 2113696...
saved file user_model/2113696
processing data for user 2161335...
saved file user_model/2161335
processing data for user 2380848...
processing data for user 30878...
saved file user_model/30878
processing data for user 52736...
saved file user_model/52736
processing data for user 584301...
saved file user_model/584301
processing data for user 956464...
saved file user_model/956464
